In [ ]:
%load_ext autoreload
%autoreload 2


# Loading Your Detections

* Use the variables below to point the Toolbox scripts at your data.
* Fill in the strings that are commented with `# REQUIRED:` then highlight a cell and hit Run Cell ('play' button above)

In [ ]:
from common_python.load_detections import loadDetections
from common_python.filter_detections import filterDetections

#  Set some variables
detection_file = ''      #  REQUIRED: And provide the filename here


version_id = '' #  OPTIONAL: Manually set a version ID number [0 for initial(first) load]
time_interval = 60 #  OPTIONAL: Time interval used to evaluate suspect detections (in minutes)

#  OPTIONAL: Average Distance from receivers, in meters, at which tags can be detected
detection_radius = '' 
#  There will be an opportunity later to change these values for individual stations
#  If entered: Value must be between 0 and 999 


#  Create a suspect detections file?
SuspectDetections = True #  True / False 

#  If you'd like to create a customizable distance matrix file for adjusting the distance 
#  between stations found in the above detection file
CreateDistanceMatrix = True #  True / False

#  If you want to reload the input file into the internal database
#  Setting this to False will match a previously loaded instance of this filename and 
#  use the data from there. For very large and unchanging input datasets, it's OK
#  to set this to False.
ReloadInputFile = True  #  True / False

#  Loads detection data into the PostgreSQL table and returns a list of table names for the tables it creates
table_names = loadDetections(detection_file, version_id, CreateDistanceMatrix, 
                   ReloadInputFile, SuspectDetections, time_interval, detection_radius) 

# Wait for output below to say 'Loading complete.'

# Filtering Detections on Distance / Time

Tables created by this script and the script above are returned by `loadDetections()` and listed in the `table_names` variable.
Types of table are Detection, station matrix, and suspect 

In [ ]:
# If you have your own file of suspect detections, point to it here 
# and set OverrideSuspectDetectionFile to True:

OverrideSuspectDetectionFile = False #  True / False


# Can replace the generated Suspect Detections file with your hand-edited file of suspect detections
# This file should live in your shared data folder.
if OverrideSuspectDetectionFile:
    # Supply the Suspect Detections file here:
    SuspectFile = 'My_Suspect_Detections_File.csv' 
else:
    SuspectFile = None

filterDetections(detection_file, version_id, SuspectFile, 
                     OverrideSuspectDetectionFile, CreateDistanceMatrix, 
                     detection_radius, ReloadInputFile)  # Wait for Out[] to say 'Filtering complete.'